In [1]:
# Install all required packages
%package install bloomberg.bquant.signal_lab=1.5.2

Running: micromamba install bloomberg.bquant.signal_lab=1.5.2 --yes --quiet --log-level=error

Note: Packages not from Bloomberg channels are not vetted by Bloomberg.
Please restart the Jupyter kernel if you run into any issues after installing or updating packages via %package.



In [2]:
import bloomberg.bquant.signal_lab as signal_lab
signal_lab.__version__

'1.5.2'

In [3]:
import bql


from bloomberg.bquant.signal_lab.workflow.node import (
    industry_grouping, portfolio_construction)
from bloomberg.bquant.signal_lab.signal.transformers import WeightingScheme
from bloomberg.bquant.signal_lab.workflow.factory import (
    UniverseFactory,
    DataItemFactory,
    SignalFactory,
)
from bloomberg.bquant.signal_lab.workflow import (
    AnalyticsDataConfig,
    build_backtest,
)

from bloomberg.bquant.signal_lab.workflow.utils import get_sandbox_path

import numpy as np
import pandas as pd

In [4]:
# Get the saved DataPack path
bq = bql.Service()
data_pack_path = f"{get_sandbox_path()}/esl/datapack_snapshot"
data_pack_path

's3://awmgd-prod-finml-sandbox-user/bclarke16/esl/datapack_snapshot'

In [5]:
import backtest_params as bp
import importlib

In [6]:
# Key backtest parameters
start = "2020-01-05"
end = "2024-03-01"
universe_name = "INDU Index"

In [7]:
universe, benchmark = bp.get_universe_params(start, end, universe_name, data_pack_path)
price, cur_mkt_cap = bp.get_return_params(start, end, data_pack_path)

In [8]:
# Import the test data set while developing
df = pd.read_csv('test_analysis.csv')

In [9]:
df = df[['Date','Security','Decision','Confidence']]

In [10]:
df

,Date,Security,Decision,Confidence
0,2020-02-06,MMM UN Equity,BUY,70.0
1,2020-02-12,CSCO UW Equity,BUY,80.0
2,2020-02-12,AMGN UQ Equity,BUY,80.0
3,2020-02-13,AXP UN Equity,BUY,80.0
4,2020-02-13,NVDA UQ Equity,BUY,80.0
...,...,...,...,...
891,2025-01-31,XOM UN Equity,BUY,70.0
892,2025-02-04,PFE UN Equity,BUY,70.0
893,2025-02-04,MRK UN Equity,BUY,80.0
894,2025-02-04,AMGN UW Equity,BUY,70.0


In [25]:
type(bq.data.id_bb_global())

bql.om.bql_item.BqlItem

In [63]:
def _bql_execute_single(univ: list[str], field: dict[str, bql.om.bql_item.BqlItem]) -> pd.DataFrame:
    """Execute a BQL query with one field"""
    req = bql.Request(univ, field)
    data = bq.execute(req)
    return data[0].df()

def convert_to_figi(df: pd.DataFrame) -> pd.DataFrame:
    """Function to convert Bloomberg tickers in a dataframe to FIGIs for ESL"""
    univ      = df['Security'].to_list()
    field     = {'figi': bq.data.composite_id_bb_global()}
    figi      = _bql_execute_single(univ, field)
    merged_df = df.merge(figi, left_on='Security', right_index=True).sort_index()
    return merged_df[['Date', 'figi', 'Decision', 'Confidence']].rename(columns={'figi':'Security'})
    

In [64]:
df1 = convert_to_figi(df)

In [68]:
list(df1['Security'].unique())

['BBG000BP52R2',
 'BBG000C3J3C9',
 'BBG000BBS2Y0',
 'BBG000BCQZS4',
 'BBG000BBJQV0',
 'BBG000CH5208',
 'BBG00BN961G4',
 'BBG000BMHYD1',
 'BBG000BF0K17',
 'BBG000HS77T5',
 'BBG000K4ND22',
 'BBG000C6CFJ5',
 'BBG000BMX289',
 'BBG000BKZB36',
 'BBG000BN2DC2',
 'BBG000BWXBC2',
 'BBG000C5HS04',
 'BBG000BWLMJ4',
 'BBG000DMBXR2',
 'BBG000BR2TH3',
 'BBG000BJ81C1',
 'BBG000C0G1D1',
 'BBG000BLNNH6',
 'BBG000BPD168',
 'BBG000BPH459',
 'BBG000BSXQV7',
 'BBG000BCSST7',
 'BBG000BNSZP1',
 'BBG000B9XRY4',
 'BBG000H556T9',
 'BBG000BVPV84',
 'BBG00BN96922',
 'BBG000PSKYX7',
 'BBG000BH4R78',
 'BBG000GZQ728',
 'BBG000BR2B91',
 'BBG000BW8S60']

In [77]:
def event_backtest(df_events: pd.DataFrame, universe: UniverseFactory, signal: SignalFactory):
    """Take a list of trades and create the weights in the portfolio"""
    converted_df = convert_to_figi(df_events)
    
    signal.bind_universe(universe)
    long_portfolio = signal.df()
    short_portfolio = long_portfolio.copy(deep=True)
    
    long_portfolio.loc[:,:] = False
    short_portfolio.loc[:,:] = False
    
    # STEP 1 get the list of securities in th df_events database
    unique_securities = list(converted_df['Security'].unique())
    
    # STEP 2: iterate over the list of securities to look at the individual trades
    for security in unique_securities:
        try:
            security_trades = converted_df[converted_df['Security'] == security]
        
            # STEP 3: iterate over the trades and update the long/ short portfolio depending on trade direction
            for row in security_trades.itertuples():
                if row.Decision == 'BUY':
                    long_portfolio[security].loc[row.Date:] = True
                    short_portfolio[security].loc[row.Date:] = False
                if row.Decision == 'SELL':
                    long_portfolio[security].loc[row.Date:] = False
                    short_portfolio[security].loc[row.Date:] = True
                if row.Decision == 'HOLD':
                    continue
                if row.Decision == 'Missing':
                    continue
        except KeyError:
            print(f"Missing: {security}")
    
    # STEP 4: return the long and short portfolios
    return long_portfolio, short_portfolio

In [78]:
lp, sp = event_backtest(df, universe, price) 

100%|██████████| 1/1 [00:05<00:00,  5.73s/it]


Missing: BBG000BBJQV0
Missing: BBG00BN961G4
Missing: BBG000BSXQV7


In [80]:
sp

ID,BBG000B9XRY4,BBG000BBS2Y0,BBG000BCQZS4,BBG000BCSST7,BBG000BF0K17,BBG000BH4R78,BBG000BJ81C1,BBG000BKZB36,BBG000BLNNH6,BBG000BMHYD1,...,BBG000C5HS04,BBG000C6CFJ5,BBG000CH5208,BBG000DMBXR2,BBG000GZQ728,BBG000H556T9,BBG000HS77T5,BBG000K4ND22,BBG000PSKYX7,BBG00BN96922
DATE,,,,,,,,,,,,,,,,,,,,,
2020-05-01,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2020-05-04,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2020-05-05,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2020-05-06,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2020-05-07,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-26,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2024-02-27,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2024-02-28,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [44]:
long_portfolio = price.df()

100%|██████████| 1/1 [00:05<00:00,  5.47s/it]


In [45]:
short_portfolio = long_portfolio.copy(deep=True)

In [46]:
long_portfolio.loc[:,:] = False
short_portfolio.loc[:,:] = False

In [71]:
df1

,Date,Security,Decision,Confidence
0,2020-02-06,BBG000BP52R2,BUY,70.0
1,2020-02-12,BBG000C3J3C9,BUY,80.0
2,2020-02-12,BBG000BBS2Y0,BUY,80.0
3,2020-02-13,BBG000BCQZS4,BUY,80.0
4,2020-02-13,BBG000BBJQV0,BUY,80.0
...,...,...,...,...
891,2025-01-31,BBG000GZQ728,BUY,70.0
892,2025-02-04,BBG000BR2B91,BUY,70.0
893,2025-02-04,BBG000BPD168,BUY,80.0
894,2025-02-04,BBG000BBS2Y0,BUY,70.0


In [73]:
df1[df1['Decision'] == 'BUY']

KeyError: False

In [66]:
unique_securities = list(df1['Security'].unique())
#unique_securities

In [74]:
for security in unique_securities:
    print(security)
    security_trades = df1[df1['Security'] == security]

    # STEP 3: iterate over the trades and update the long/ short portfolio depending on trade direction
    for row in security_trades.itertuples():
        if row.Decision == 'BUY':
            long_portfolio[security].loc[row.Date:] = True
            short_portfolio[security].loc[row.Date:] = False
        if row.Decision == 'SELL':
            long_portfolio[security].loc[row.Date:] = False
            short_portfolio[security].loc[row.Date:] = True
        if row.Decision == 'HOLD':
            continue
        if row.Decision == 'Missing':
            continue

BBG000BP52R2
BBG000C3J3C9
BBG000BBS2Y0
BBG000BCQZS4
BBG000BBJQV0


KeyError: 'BBG000BBJQV0'

In [16]:
universe.df()

100%|██████████| 1/1 [00:03<00:00,  3.56s/it]


ID,BBG000B9XRY4,BBG000BBS2Y0,BBG000BCQZS4,BBG000BCSST7,BBG000BF0K17,BBG000BH4R78,BBG000BJ81C1,BBG000BKZB36,BBG000BLNNH6,BBG000BMHYD1,...,BBG000C5HS04,BBG000C6CFJ5,BBG000CH5208,BBG000DMBXR2,BBG000GZQ728,BBG000H556T9,BBG000HS77T5,BBG000K4ND22,BBG000PSKYX7,BBG00BN96922
DATE,,,,,,,,,,,,,,,,,,,,,
2020-05-01,1,0,1,1,1,1,1,1,1,1,...,1,1,1,1,1,0,1,1,1,1
2020-05-04,1,0,1,1,1,1,1,1,1,1,...,1,1,1,1,1,0,1,1,1,1
2020-05-05,1,0,1,1,1,1,1,1,1,1,...,1,1,1,1,1,0,1,1,1,1
2020-05-06,1,0,1,1,1,1,1,1,1,1,...,1,1,1,1,1,0,1,1,1,1
2020-05-07,1,0,1,1,1,1,1,1,1,1,...,1,1,1,1,1,0,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-26,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,0,1,1,1,1,1
2024-02-27,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,0,1,1,1,1,1
2024-02-28,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,0,1,1,1,1,1


In [19]:
# modify the price signal
price.bind_universe(universe)
price_df = price.df()

100%|██████████| 1/1 [00:05<00:00,  5.42s/it]


In [34]:
price_df.loc[:,:] = False

In [58]:
secs = list(price_df.columns)

In [62]:
req = bql.Request(secs, bq.data.name())
data = bq.execute(req)
data[0].df() # BBG000BP52R2

,NAME()
ID,
BBG000B9XRY4,Apple Inc
BBG000BBS2Y0,Amgen Inc
BBG000BCQZS4,American Express Co
BBG000BCSST7,Boeing Co/The
BBG000BF0K17,Caterpillar Inc
BBG000BH4R78,Walt Disney Co/The
BBG000BJ81C1,Travelers Cos Inc/The
BBG000BKZB36,Home Depot Inc/The
BBG000BLNNH6,International Business Machine


In [59]:
price_df['BBG000BCQZS4'].loc['2024-02-28':] = False


In [39]:
price_df

ID,BBG000B9XRY4,BBG000BBS2Y0,BBG000BCQZS4,BBG000BCSST7,BBG000BF0K17,BBG000BH4R78,BBG000BJ81C1,BBG000BKZB36,BBG000BLNNH6,BBG000BMHYD1,...,BBG000C5HS04,BBG000C6CFJ5,BBG000CH5208,BBG000DMBXR2,BBG000GZQ728,BBG000H556T9,BBG000HS77T5,BBG000K4ND22,BBG000PSKYX7,BBG00BN96922
DATE,,,,,,,,,,,,,,,,,,,,,
2020-05-01,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2020-05-04,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2020-05-05,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2020-05-06,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2020-05-07,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-26,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2024-02-27,False,False,True,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2024-02-28,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
